In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

더 정교한 YOLO 사용

In [ ]:
from ultralytics import YOLO
import cv2

# 1. 더 정확한 YOLOv8s 모델 로드
model = YOLO("yolov8s.pt")  # yolov8n.pt 보다 더 정밀함
COCO_CLASSES = model.names

# 2. 영상 불러오기
video_path = "pitch.mp4"  # 영상 파일이 /content/pitch.mp4 에 있어야 함
cap = cv2.VideoCapture(video_path)

# 3. 결과 영상 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("pitch_ball_yolov8s.mp4", fourcc, fps, (width, height))

# 4. 프레임 순회
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8s 객체 탐지
    results = model(frame, verbose=False)

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
        classes = r.boxes.cls.cpu().numpy()

        for box, cls_id in zip(boxes, classes):
            class_name = COCO_CLASSES[int(cls_id)]

            # 공(sports ball)만 탐지해서 파란 박스로 표시
            if class_name == "sports ball":
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 4)

    # 결과 프레임 저장
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 중단 가능
        break

# 5. 자원 해제
cap.release()
out.release()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


칼만예측을 이용한 야구공 궤도 추적 및 파울 판정

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import deque
import math

# =============================
# 사용자 파라미터(영상/구장 구도 맞춰 조절)
# =============================
VIDEO_PATH = "pitch.mp4"
OUT_PATH   = "pitch_foul_predict.mp4"

# YOLO / 탐지
IMG_SIZE    = 1280
CONF_THRES  = 0.30
IOU_THRES   = 0.50

# 파울 스트립(좌/우 가장자리)
FOUL_STRIPE_W = 0.18

# '흰 공' 전용 2차 필터
WHITE_S_MAX       = 70     # HSV 채도 상한(흰색은 낮음)
WHITE_V_MIN       = 160    # HSV 밝기 하한(흰색은 밝음)
WHITE_RATIO_MIN   = 0.22   # 박스 중앙부 흰색 비율 하한
CIRC_MIN          = 0.35   # 원형 지수 하한(0~1)

# 박스 크기/종횡비/속도 1차 필터
AREA_MIN_RATIO = 0.00002
AREA_MAX_RATIO = 0.01
ASPECT_MAX     = 1.6
MIN_SPEED_PX   = 2.0

# 추적/표시
HISTORY_LEN       = 20
DRAW_PRED_STEPS   = 30        # 칼만 예측 궤적 점 개수
DRAW_SIMPLE_LINEAR= True      # ✔ 간단 선형 예측(최근 속도로 직선) 사용할지
POS_HISTORY_LEN   = 10        # 선형 예측을 위한 최근 좌표 저장 길이
LINEAR_STEPS      = 5         # 선형 예측 프레임 수

# 타자 시점 라벨 전환(기하학은 영상 기준 고정)
BATTER_VIEW = False

# 구도 수동 지정(필요시 True로 하고 좌표 입력)
USE_MANUAL_POLES = False
HOME_PX_MANUAL   = None  # (x, y)
LEFT_POLE_MANUAL = None  # (x, y)
RIGHT_POLE_MANUAL= None  # (x, y)

# ROI 추가 탐지
ROI_ENABLE = True
ROI_SCALE  = 1.6
ROI_MIN    = 160
ROI_MAX    = 640

# =============================
# Kalman Filter (x, y, vx, vy)
# =============================
class BallKF:
    def __init__(self, dt=1.0):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.transitionMatrix = np.array([[1,0,dt,0],
                                             [0,1,0,dt],
                                             [0,0,1, 0],
                                             [0,0,0, 1]], dtype=np.float32)
        self.kf.measurementMatrix = np.array([[1,0,0,0],
                                              [0,1,0,0]], dtype=np.float32)
        self.kf.processNoiseCov  = np.eye(4, dtype=np.float32) * 1e-2
        self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1e-1
        self.kf.errorCovPost = np.eye(4, dtype=np.float32)
        self.initialized = False

    def init(self, x, y):
        self.kf.statePost = np.array([[x],[y],[0.0],[0.0]], dtype=np.float32)
        self.initialized = True

    def predict(self):
        return self.kf.predict()

    def correct(self, x, y):
        z = np.array([[x],[y]], dtype=np.float32)
        return self.kf.correct(z)

    def state(self):
        return self.kf.statePost.copy()

# =============================
# 보조 유틸
# =============================
def clamp(v, lo, hi): return max(lo, min(hi, v))

def angle_of(vec):
    """오른쪽=0°, 위쪽=+90° (영상 y는 아래로 + → -dy)."""
    dx, dy = vec
    return math.degrees(math.atan2(-dy, dx))

def normalize_angle(a):
    return (a + 180) % 360 - 180

def line_angle(p0, p1):
    v = (p1[0]-p0[0], p1[1]-p0[1])
    return angle_of(v)

def fair_foul_reason(launch_deg, left_deg, right_deg, batter_view=False):
    """
    콘솔 설명: 발사각/좌·우 폴 각도 비교 + 타자 기준 라벨(폴 좌표는 영상 기준 고정)
    """
    def norm(a): return (a + 180) % 360 - 180
    la = norm(left_deg);  ra = norm(right_deg)

    # 라벨만 타자 기준으로 전환
    if batter_view:
        left_label  = "오른쪽(타자 기준)"
        right_label = "왼쪽(타자 기준)"
    else:
        left_label  = "왼쪽(타자 기준)"
        right_label = "오른쪽(타자 기준)"

    span = norm(ra - la)
    if span < 0:
        la, ra = ra, la
        span = norm(ra - la)

    t = norm(launch_deg - la)
    if 0 <= t <= span:
        msg = (f"발사각 {launch_deg:.1f}°, 좌폴 {left_deg:.1f}°, 우폴 {right_deg:.1f}° → "
               f"두 폴 사이이므로 foul이 아닙니다 (타자 시점={batter_view}).")
        return True, msg
    else:
        side = left_label if t < 0 else right_label
        msg = (f"발사각 {launch_deg:.1f}°, 좌폴 {left_deg:.1f}°, 우폴 {right_deg:.1f}° → "
               f"{side} 폴 방향으로 벗어나 foul 입니다 (타자 시점={batter_view}).")
        return False, msg

# ---- '흰 공' 색/형태 필터 ----
def is_white_round_ball(frame, box):
    H, W = frame.shape[:2]
    x1,y1,x2,y2 = [int(clamp(v, 0, lim)) for v,lim in zip(box, [W-1,H-1,W,H])]
    if x2 <= x1 or y2 <= y1:
        return False
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return False

    # 중앙부만 평가(배경 영향 감소)
    h, w = roi.shape[:2]
    cx1 = int(w*0.25); cy1 = int(h*0.25)
    cx2 = int(w*0.75); cy2 = int(h*0.75)
    core = roi[cy1:cy2, cx1:cx2]
    if core.size == 0:
        core = roi

    hsv = cv2.cvtColor(core, cv2.COLOR_BGR2HSV)
    _, Sc, Vc = cv2.split(hsv)
    white_mask = (Sc <= WHITE_S_MAX) & (Vc >= WHITE_V_MIN)
    white_ratio = float(np.count_nonzero(white_mask)) / float(white_mask.size)
    if white_ratio < WHITE_RATIO_MIN:
        return False

    # 원형성(4πA/P^2)
    wm = (white_mask.astype(np.uint8) * 255)
    wm = cv2.medianBlur(wm, 3)
    cnts, _ = cv2.findContours(wm, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return False
    c = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(c)
    peri = cv2.arcLength(c, True)
    if peri <= 1e-3:
        return False
    circularity = 4.0 * np.pi * area / (peri*peri)
    return circularity >= CIRC_MIN

def pick_best_ball(frame, det_boxes, det_confs, frame_shape, pred_xy=None, gate_dist=80):
    H, W = frame_shape[:2]
    area_min = AREA_MIN_RATIO * (W*H)
    area_max = AREA_MAX_RATIO * (W*H)

    candidates = []
    for box, conf in zip(det_boxes, det_confs):
        x1,y1,x2,y2 = box.astype(int)
        w = max(1, x2-x1); h = max(1, y2-y1)
        area = w*h
        aspect = w/h if w>h else h/w

        # 1차(크기/종횡비)
        if area < area_min or area > area_max:
            continue
        if aspect > ASPECT_MAX:
            continue

        # 2차(흰 공/원형)
        if not is_white_round_ball(frame, (x1,y1,x2,y2)):
            continue

        cx = (x1 + x2)//2
        cy = (y1 + y2)//2
        dist = np.linalg.norm(np.array([cx,cy]) - np.array(pred_xy)) if pred_xy is not None else 0.0
        candidates.append((conf, dist, (cx,cy), (x1,y1,x2,y2)))

    if not candidates:
        return None

    if pred_xy is not None:
        gated = [c for c in candidates if c[1] <= gate_dist]
        pool = gated if gated else candidates
    else:
        pool = candidates

    pool.sort(key=lambda x: (-x[0], x[1]))
    return pool[0]

def run_yolo_on_roi(model, frame, center_xy, imgsz, conf, iou):
    if center_xy is None:
        return [], []
    H, W = frame.shape[:2]
    cx, cy = int(center_xy[0]), int(center_xy[1])

    side = int(clamp(min(W, H) / 4 * ROI_SCALE, ROI_MIN, ROI_MAX))
    x1 = clamp(cx - side//2, 0, W-1)
    y1 = clamp(cy - side//2, 0, H-1)
    x2 = clamp(x1 + side, 1, W)
    y2 = clamp(y1 + side, 1, H)

    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return [], []

    r = model.predict(
        roi,
        imgsz=imgsz, conf=conf, iou=iou,
        classes=[32],  # sports ball
        verbose=False,
        agnostic_nms=True,
        max_det=20
    )[0]

    boxes_roi, confs_roi = [], []
    if r.boxes is not None and len(r.boxes) > 0:
        b = r.boxes.xyxy.cpu().numpy()
        c = r.boxes.conf.cpu().numpy()
        for bb, cc in zip(b, c):
            bx1, by1, bx2, by2 = bb
            boxes_roi.append(np.array([bx1 + x1, by1 + y1, bx2 + x1, by2 + y1], dtype=np.float32))
            confs_roi.append(float(cc))

    return boxes_roi, confs_roi

# =============================
# 메인
# =============================
def main():
    # 모델 로드 & sports ball 클래스 ID
    model = YOLO("yolov8s.pt")
    names = model.names
    if isinstance(names, dict):
        inv = {v:k for k,v in names.items()}
        sports_ball_class_id = int(inv.get("sports ball", 32))
    else:
        sports_ball_class_id = names.index("sports ball") if "sports ball" in names else 32

    # 비디오 입출력
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise RuntimeError(f"비디오를 열 수 없습니다: {VIDEO_PATH}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUT_PATH, fourcc, fps, (W, H))

    # 홈/폴 좌표(영상 기준, 시점 전환은 라벨만)
    if USE_MANUAL_POLES and HOME_PX_MANUAL and LEFT_POLE_MANUAL and RIGHT_POLE_MANUAL:
        home_px_scr        = (int(HOME_PX_MANUAL[0]),   int(HOME_PX_MANUAL[1]))
        left_pole_px_view  = (int(LEFT_POLE_MANUAL[0]), int(LEFT_POLE_MANUAL[1]))
        right_pole_px_view = (int(RIGHT_POLE_MANUAL[0]),int(RIGHT_POLE_MANUAL[1]))
    else:
        home_px_scr        = (int(W*0.5), int(H*0.95))
        left_pole_px_view  = (0, 0)          # 화면 왼쪽 폴
        right_pole_px_view = (W-1, 0)        # 화면 오른쪽 폴

    left_ang_view  = line_angle(home_px_scr, left_pole_px_view)
    right_ang_view = line_angle(home_px_scr, right_pole_px_view)

    # 추적/상태
    kf = BallKF(dt=1.0)
    history = deque(maxlen=HISTORY_LEN)
    lost_frames = 0
    avg_speed   = 0.0
    decision_printed = False

    # ✔ 간단 선형 예측용 최근 좌표
    positions = deque(maxlen=POS_HISTORY_LEN)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 칼만 예측
        pred_xy = None
        if kf.initialized:
            pred = kf.predict()
            px, py, pvx, pvy = pred.flatten()
            pred_xy = (px, py)

        # 전체 프레임 탐지
        results = model.predict(
            frame,
            imgsz=IMG_SIZE,
            conf=CONF_THRES,
            iou=IOU_THRES,
            classes=[sports_ball_class_id],
            verbose=False,
            agnostic_nms=True,
            max_det=20
        )

        det_boxes, det_confs = [], []
        if results and len(results)>0:
            r = results[0]
            if r.boxes is not None and len(r.boxes)>0:
                det_boxes.extend(list(r.boxes.xyxy.cpu().numpy()))
                det_confs.extend(list(r.boxes.conf.cpu().numpy()))

        # ROI 추가 탐지
        if ROI_ENABLE and pred_xy is not None:
            roi_boxes, roi_confs = run_yolo_on_roi(
                model, frame, pred_xy, IMG_SIZE, CONF_THRES, IOU_THRES
            )
            if roi_boxes:
                det_boxes.extend(roi_boxes)
                det_confs.extend(roi_confs)

        # 동적 게이트(속도에 따라 60~140px)
        gate = clamp(70.0 + avg_speed*1.2, 60.0, 140.0)

        selection = None
        if len(det_boxes) > 0:
            selection = pick_best_ball(frame, det_boxes, det_confs, frame.shape, pred_xy=pred_xy, gate_dist=gate)

        if selection is not None:
            conf, dist, (cx,cy), (x1,y1,x2,y2) = selection
            if not kf.initialized:
                kf.init(cx, cy)
            else:
                kf.correct(cx, cy)

            state = kf.state().flatten()
            x, y, vx, vy = state
            history.append((int(x), int(y)))
            lost_frames = 0

            # 시각화(박스/점)
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0), 2)
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

            # ✔ 간단 선형 예측용 최근 좌표 업데이트
            positions.append((int(x), int(y)))
        else:
            if kf.initialized:
                state = kf.state().flatten()
                x, y, vx, vy = state
                history.append((int(x), int(y)))
                lost_frames += 1

                # ✔ 탐지 실패 시에도 선형 예측용 최근 좌표 유지(추정값 사용 X → 최근 관측 기반만)
            else:
                lost_frames += 1

        # 너무 오래 잃으면 리셋
        if lost_frames > int(fps*1.5):
            kf = BallKF(dt=1.0)
            history.clear()
            positions.clear()
            lost_frames = 0
            avg_speed = 0.0

        # 궤적 선
        for i in range(1, len(history)):
            cv2.line(frame, history[i-1], history[i], (0,255,0), 2)

        # 칼만 예측 궤적(주황 점)
        if kf.initialized and len(history)>=3:
            state = kf.state().flatten()
            sim = state.copy()
            for _ in range(DRAW_PRED_STEPS):
                F = kf.kf.transitionMatrix
                sim = (F @ sim.reshape(-1,1)).flatten()
                cv2.circle(frame, (int(sim[0]), int(sim[1])), 2, (0,165,255), -1)

        # ✔ 간단 선형 예측(빨간 점)
        if DRAW_SIMPLE_LINEAR and len(positions) >= 5:
            pts = np.array(positions)
            dx = pts[-1][0] - pts[-2][0]
            dy = pts[-1][1] - pts[-2][1]
            future_pts = [(pts[-1][0] + dx * i, pts[-1][1] + dy * i) for i in range(1, LINEAR_STEPS+1)]
            for pt in future_pts:
                cv2.circle(frame, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)

        # 평균 속도 갱신
        if kf.initialized and len(history) >= 5:
            spds = [np.linalg.norm(np.array(history[i]) - np.array(history[i-1]))
                    for i in range(1, len(history))]
            avg_speed = float(np.mean(spds))

        # 좌/우 반투명 스트립
        left_x  = int(FOUL_STRIPE_W * W)
        right_x = int((1.0 - FOUL_STRIPE_W) * W)
        overlay = frame.copy()
        cv2.rectangle(overlay, (0,0), (left_x,H), (0,0,255), -1)
        cv2.rectangle(overlay, (right_x,0), (W,H), (0,0,255), -1)
        frame = cv2.addWeighted(overlay, 0.10, frame, 0.90, 0)

        # ===== 콘솔에 1회 파울/페어 판정 출력 =====
        if (not decision_printed) and kf.initialized and len(history) >= 6 and avg_speed >= MIN_SPEED_PX:
            state = kf.state().flatten()
            vx, vy = state[2], state[3]
            launch_angle = angle_of((vx, vy))
            is_fair, reason_text = fair_foul_reason(
                launch_angle, left_ang_view, right_ang_view, batter_view=BATTER_VIEW
            )
            print(reason_text)
            print("foul이 아닙니다" if is_fair else "foul 입니다.")
            decision_printed = True

        # 저장/종료 키
        out.write(frame)

    cap.release()
    out.release()

    if not decision_printed:
        print("충분한 궤적/속도 추정이 없어 판정을 출력하지 못했습니다.")
    print(f"완료: {OUT_PATH}")

if __name__ == "__main__":
    main()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


발사각 -89.9°, 좌폴 118.8°, 우폴 61.3° → 왼쪽(타자 기준) 폴 방향으로 벗어나 foul 입니다 (타자 시점=False).
foul 입니다.
완료: pitch_foul_predict.mp4
